# Dada las necesidades especificas de enriquecimiento de data, desde el punto de vista de informacion colombiana, optamos por realizar un scrapper a paginas amarillas donde podamos enriquecer los nouns identificados

In [2]:
import pandas as pd
import requests
import unicodedata
#Para instalar bs4: pip install beautifulsoup4
from bs4 import BeautifulSoup

baseurl='https://www.paginasamarillas.com.co/servicios/'

def search(keyword):
    r = requests.get(baseurl+keyword)
    if r.status_code < 300:
        soup = BeautifulSoup(r.content, 'html.parser')
        del r
        cant_resultados = soup.find('h1',class_='light').get_text().split()[-2]
        if cant_resultados is None:
            return False
        resultados = soup.find_all('div', class_='figBox')

        info_list = []
        activities_list = []
        for i in range(1):
            result = resultados[i]

            res_url = result.find('a',href=True)['href']

            result_r = requests.get(res_url)
            
            result_soup = BeautifulSoup(result_r.content, 'html.parser')
            del result_r
            activities_soup = result_soup.find('p', class_='bip-activities')

            #for activity in activities_soup.find_all('a'):
            
            if activities_soup is None:
                #activities_list.append([])
                pass
            else:
                for activity in activities_soup.find_all('a'):
                    activities_list.append(unicodedata.normalize('NFKD',activity.get_text().lower()).encode('ascii', 'ignore').decode())
               
            
        return activities_list
    

In [3]:
data = pd.read_pickle('dt_trxpse_V3.pkl.gzip')

In [21]:
%timeit search('referencia')

3.66 s ± 885 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# En este momento es inviable usar esto en los 11M de transacciones.

# El tiempo de computo escalaria aprox a 3.66 * 450000 segs.

# Vale la pena resaltar que seria la manera ideal de enriquecer el lenguaje. Esto se debe a que la palabra esta sujeta a un contexto colombiano, donde las categorias son lexico que tendria muchisimo potencial predictivo para categorizar.

In [22]:
search('avianca')

['turismo', 'viajes', 'agencias de viajes']

In [23]:
search('eticket')

['boleteria para espectaculos']

In [24]:
search('pago')

['diligencias y tramites', 'mensajeros', 'servicios a domicilio']

In [25]:
search('pensiones')

['abogados - asuntos civiles',
 'abogados - asuntos criminales',
 'abogados - asuntos de seguros']

In [28]:
search('cuidadania')

['centros comerciales', 'comercio', 'centros de belleza y estetica']

In [29]:
search('booking')

['turismo']

In [32]:
search('falabella')

['bonos de incentivos', 'cambio de cheques', 'prestamos']

In [34]:
search('cmr')

['software financiero',
 'software de nomina y recursos humanos',
 'consultoria en software']

In [53]:
search_list = []
for row in data['ref1_exp_nlp']:
    if isinstance(row, list):
        for double_row in row:
            search_list.append(double_row)
    else:
        search_list.append(row)

In [54]:
pd.Series(search_list).unique().size

346211

# Exactamente se necesitan 346211 busquedas unicas para enriquecer esta base.